# Carga de librerias y datos

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

udea-ai-4-eng-20251-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline
dtr= pd.read_csv("train.csv")

# PREPROCESAMIENTO

## DATOS BOOLEANOS

In [ ]:
def mask(x):
  if x=="Si":
    return bool(True)
  elif x=="No":
    return bool(False)
  else:
    return
#Se define una mascara para que los datos queden en el tipo adecuado

In [ ]:
for i in ['FAMI_TIENEINTERNET','FAMI_TIENELAVADORA','FAMI_TIENEAUTOMOVIL','ESTU_PAGOMATRICULAPROPIO','FAMI_TIENECOMPUTADOR','FAMI_TIENEINTERNET.1']:
  dtr[i]=dtr[i].apply(mask)

In [ ]:
del dtr['FAMI_TIENEINTERNET.1'] #Observamos que es una variable repetida por lo cual borramos una de las columnas

In [ ]:
def mask1(x):
  if x=="S":
    return bool(True)
  elif x=="N":
    return bool(False)
  else:
    return
#Mascara para variable particular

In [ ]:
dtr['ESTU_PRIVADO_LIBERTAD']=dtr['ESTU_PRIVADO_LIBERTAD'].apply(mask1)

In [ ]:
bool_var= ['FAMI_TIENEINTERNET','FAMI_TIENELAVADORA','FAMI_TIENEAUTOMOVIL','ESTU_PAGOMATRICULAPROPIO','FAMI_TIENECOMPUTADOR','ESTU_PRIVADO_LIBERTAD']

In [ ]:
for col in bool_var:
  dtr[col]=dtr[col].fillna(dtr[col].mode(dropna=True)[0])
  #Al ser datos booleanos, y la cantidad de faltantes no muy alta, se decide llenar los datos faltantes con la moda de cada una de las columnas

/tmp/ipython-input-10-3536273753.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dtr[col]=dtr[col].fillna(dtr[col].mode(dropna=True)[0])
/tmp/ipython-input-10-3536273753.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dtr[col]=dtr[col].fillna(dtr[col].mode(dropna=True)[0])


## Datos categoricos mapeables

Hay una buena cantidad de columnas que tienen una estructura ordenada por lo cual para simplicidad de los datos podriamos mapearlos en integrers ordenadas, en ciertos casos se toma la decisión de llenar los valores faltantes con 0, la media, o una categoría nueva segun tenga sentido.

In [ ]:
mapa_matricula = {
    'Menos de 500 mil': 1,
    'Entre 500 mil y menos de 1 millón': 2,
    'Entre 1 millón y menos de 2.5 millones': 3,
    'Entre 2.5 millones y menos de 4 millones': 4,
    'Entre 4 millones y menos de 5.5 millones': 5,
    'Entre 5.5 millones y menos de 7 millones': 6,
    'Más de 7 millones': 7,
    'No pagó matrícula': 0,
}
dtr['ESTU_VALORMATRICULAUNIVERSIDAD'] = dtr['ESTU_VALORMATRICULAUNIVERSIDAD'].map(mapa_matricula)

In [ ]:
dtr['ESTU_VALORMATRICULAUNIVERSIDAD']=dtr['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(round(dtr['ESTU_VALORMATRICULAUNIVERSIDAD'].mean()))

In [ ]:
mapa_trabajo = {
    '0': 0,
    'Menos de 10 horas': 1,
    'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 3,
    'Más de 30 horas': 4
}

dtr['ESTU_HORASSEMANATRABAJA'] = dtr['ESTU_HORASSEMANATRABAJA'].map(mapa_trabajo)
dtr['ESTU_HORASSEMANATRABAJA'] = dtr['ESTU_HORASSEMANATRABAJA'].fillna(round(dtr['ESTU_HORASSEMANATRABAJA'].mean()))

In [ ]:
dtr['FAMI_ESTRATOVIVIENDA']=dtr['FAMI_ESTRATOVIVIENDA'].fillna('ns/nr')

In [ ]:
for col in ['FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE']:
  dtr[col]=dtr[col].fillna('desconocido')

In [ ]:
dtr.isnull().any()

,0
ID,False
PERIODO,False
ESTU_PRGM_ACADEMICO,False
ESTU_PRGM_DEPARTAMENTO,False
ESTU_VALORMATRICULAUNIVERSIDAD,False
ESTU_HORASSEMANATRABAJA,False
FAMI_ESTRATOVIVIENDA,False
FAMI_TIENEINTERNET,False
FAMI_EDUCACIONPADRE,False
FAMI_TIENELAVADORA,False


In [ ]:
Var_categoricas=['FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE','FAMI_ESTRATOVIVIENDA','ESTU_PRGM_ACADEMICO','ESTU_PRGM_DEPARTAMENTO']
for col in Var_categoricas:
  dtr[col]=dtr[col].astype(str)

In [ ]:
del dtr['coef_1'], dtr['coef_2'], dtr['coef_3'], dtr['coef_4']

#Entrenamiento,

In [ ]:
dtr.shape

(692500, 16)

In [ ]:
dtr_copy=dtr.copy()
le=LabelEncoder()
y=le.fit_transform(dtr_copy['RENDIMIENTO_GLOBAL'])
X=dtr_copy.drop(columns=["RENDIMIENTO_GLOBAL", "ID", "PERIODO"] )
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [ ]:
var_map = ['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA']
#Se establece lista de las variables mapeadas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), Var_categoricas),
        ('scaler', StandardScaler(), var_map)
    ],
    remainder='passthrough'
)
#Se escalan las variables numericas y las varibales categoricas se filtran en onehot encoder
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Despues del preprocesado se fijan los datos en el split




In [ ]:
#Entrenamos el modelo
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier",RandomForestClassifier(n_estimators=2000, max_depth=8, max_features='sqrt', random_state=68, n_jobs=-1, verbose=1))
])
pipeline.fit(x_train, y_train)

y_val_pred = pipeline.predict(x_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"Accuracy validación: {val_accuracy:.4f}")
print(classification_report(y_val, y_val_pred, target_names=le.classes_))

y_test_pred = pipeline.predict(x_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Accuracy test: {test_accuracy:.4f}")
print(classification_report(y_test, y_test_pred, target_names=le.classes_))


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   42.7s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 2000 out of 2000 | elapsed:  6.4min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    1.2s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    2.8s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    5.0s
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:    9.1s
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed:   12.7s
[Parallel(n_jobs=2)]: Do

Accuracy validación: 0.3837
              precision    recall  f1-score   support

        alto       0.45      0.64      0.52     35404
        bajo       0.37      0.63      0.47     34360
  medio-alto       0.30      0.12      0.17     34278
  medio-bajo       0.30      0.14      0.19     34458

    accuracy                           0.38    138500
   macro avg       0.35      0.38      0.34    138500
weighted avg       0.36      0.38      0.34    138500



[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.3s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    1.2s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    2.8s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    5.6s
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:    9.1s
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed:   12.5s


Accuracy test: 0.3844
              precision    recall  f1-score   support

        alto       0.45      0.64      0.53     35165
        bajo       0.38      0.63      0.47     34573
  medio-alto       0.30      0.12      0.17     34259
  medio-bajo       0.29      0.14      0.19     34503

    accuracy                           0.38    138500
   macro avg       0.35      0.38      0.34    138500
weighted avg       0.35      0.38      0.34    138500



[Parallel(n_jobs=2)]: Done 2000 out of 2000 | elapsed:   13.8s finished


Ahora para la subida a Kaggle debemos adaptar los datos de test a los mapas establecidos.

In [ ]:
dtest= pd.read_csv("test.csv")

In [ ]:
for i in ['FAMI_TIENEINTERNET','FAMI_TIENELAVADORA','FAMI_TIENEAUTOMOVIL','ESTU_PAGOMATRICULAPROPIO','FAMI_TIENECOMPUTADOR','FAMI_TIENEINTERNET.1']:
  dtest[i]=dtest[i].apply(mask)
del dtest['FAMI_TIENEINTERNET.1']
dtest['ESTU_PRIVADO_LIBERTAD']=dtest['ESTU_PRIVADO_LIBERTAD'].apply(mask1)
for col in bool_var:
  dtest[col]=dtest[col].fillna(dtest[col].mode(dropna=True)[0])
dtest['ESTU_VALORMATRICULAUNIVERSIDAD'] = dtest['ESTU_VALORMATRICULAUNIVERSIDAD'].map(mapa_matricula)
dtest['ESTU_VALORMATRICULAUNIVERSIDAD']=dtest['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna(round(dtest['ESTU_VALORMATRICULAUNIVERSIDAD'].mean()))
dtest['ESTU_HORASSEMANATRABAJA'] = dtest['ESTU_HORASSEMANATRABAJA'].map(mapa_trabajo)
dtest['ESTU_HORASSEMANATRABAJA'] = dtest['ESTU_HORASSEMANATRABAJA'].fillna(round(dtr['ESTU_HORASSEMANATRABAJA'].mean()))
dtest['FAMI_ESTRATOVIVIENDA']=dtest['FAMI_ESTRATOVIVIENDA'].fillna('ns/nr')
for col in ['FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE']:
  dtest[col]=dtest[col].fillna('desconocido')
Var_categoricas=['FAMI_EDUCACIONMADRE','FAMI_EDUCACIONPADRE','FAMI_ESTRATOVIVIENDA','ESTU_PRGM_ACADEMICO','ESTU_PRGM_DEPARTAMENTO']
for col in Var_categoricas:
  dtest[col]=dtest[col].astype(str)
del dtest['coef_1'], dtest['coef_2'], dtest['coef_3'], dtest['coef_4']

/tmp/ipython-input-24-2315766531.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dtest[col]=dtest[col].fillna(dtest[col].mode(dropna=True)[0])


In [ ]:
x_kaggle = dtest.drop(columns=["ID", "PERIODO"], errors='ignore')
predictions = pipeline.predict(x_kaggle)
predictions_labels = le.inverse_transform(predictions)
kaggle_dtest = dtest[["ID"]].copy()
kaggle_dtest["RENDIMIENTO_GLOBAL"] = predictions_labels
kaggle_dtest.to_csv("submission_dtest.csv", index=False)
print(kaggle_dtest.head())

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    1.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    4.9s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    9.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:   15.5s
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:   22.0s
[Parallel(n_jobs=2)]: Done 1796 tasks      | elapsed:   31.2s
[Parallel(n_jobs=2)]: Done 2000 out of 2000 | elapsed:   34.1s finished


       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545         medio-alto
2  499179               alto
3  782980               bajo
4  785185         medio-bajo
